In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from skimage.exposure import equalize_hist
from skimage import io, color
from skimage.transform import rotate
from skimage.transform import resize
from skimage.transform import rescale
from skimage.exposure import adjust_gamma
from sklearn.model_selection import StratifiedShuffleSplit
from numpy.linalg import svd
from scipy.optimize import minimize
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# Define paths to the folders containing images
boxer_folder = 'boxer'
persian_cat_folder = 'persian_cat'

#image_size = (256, 256)  # Resize all images to this size
image_size = (64, 64) 

# Function to load and preprocess images from a folder
def load_images(folder_path, image_size):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.JPG', '.jpeg')):
            img = io.imread(os.path.join(folder_path, filename))

            # Ensure the image is RGB (3 channels) by removing an alpha channel if it exists
            if img.shape[-1] == 4:
                img = img[..., :3]  # Keep only the first 3 channels (RGB)

            img_gray = color.rgb2gray(img)  # Convert to grayscale
            img_resized = resize(img_gray, image_size)  # Resize
            images.append(img_resized)  # Store resized image
    return np.array(images)  # converts the list of images to a NumPy array